In [ ]:
import pandas as pd
import numpy as py
import matplotlib.pyplot as plt


In [ ]:
# Teammate 0 code begins here
# changes here
state_name_list = [ "Alabama", "Alaska", "Arizona", "Arkansas", "California",
                    "Colorado", "Connecticut", "DC", "Delaware", "Florida",
                    "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana",
                    "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine",
                    "Maryland", "Massachusetts", "Michigan", "Minnesota", 
                    "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada",
                    "New_Hampshire", "New_Jersey", "New_Mexico", "New_York", "North_Carolina",
                    "North_Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania",
                    "Rhode_Island", "South_Carolina", "South_Dakota", "Tennessee", "Texas",
                    "Utah", "Vermont", "Virginia", "Washington", "West_Virginia", 
                    "Wisconsin", "Wyoming"
                  ]

# An empty collection of dataframes
crime_by_state = {}


debug = False


print("Importing *.csv files...")

# Base file path we will use for reusable code
base_file_path = "Raw_Data/CrimeStateByState_"

# Import the U.S. crime data totals
file_path = base_file_path + "US" + ".csv"
if (debug):
    print(f"Importing file: {file_path}")
us_totals_df = pd.read_csv(file_path, header=5)

# Iterate over and import the state-by-state crime data
for state in state_name_list:
    file_path = base_file_path + state + ".csv"
    if (debug):
        print(f"Importing file: {file_path}")
    crime_by_state[state] = pd.read_csv(file_path, header=5)

# Import the gun ownership data
file_path = "Raw_Data/gun_ownership_by_state.csv"
if (debug):
    print(f"Importing file: {file_path}")
gun_ownership_df = pd.read_csv(file_path)

print("File import complete.")

# Clean up the footer information that was at the bottom of each crime data .csv file
if debug:
    print("Cleaning up the .csv files...")
    
for state in state_name_list:
    if debug:
        print(f"The current state is {state}...")
    crime_by_state[state] = crime_by_state[state][crime_by_state[state]["Year"].astype(str).str.isnumeric()]
    
if debug:
    print(f"The current state is {state}...")
us_totals_df = us_totals_df[us_totals_df["Year"].astype(str).str.isnumeric()]

if debug:
    print("Cleanup of .csv files is complete.")

In [ ]:
# Reindex the gun ownership data
try:
    gun_ownership_df.set_index("State", inplace=True, drop=False)
except:
    pass


In [ ]:
all_state_crime_2014 = pd.DataFrame()

# Build a dataframe of the 2014 crime data of all the states and index it to the state name
for state in state_name_list:

    if debug:
        print(f"The current state is {state}...")
    # Give each row the state name so that it can be used as an index later
    crime_by_state[state]["State"] = state
    # Use "this_state" as short-hand
    this_state = crime_by_state[state]
    # Grab the specific row we want
    this_state_crime_2014 = this_state[this_state["Year"] == "2014"]
    # We do this because .append() doesn't support "inplace"
    all_state_crime_2014 = all_state_crime_2014.append(this_state_crime_2014)
    

all_state_crime_2014.set_index(["State"], inplace=True)

try:
    gun_ownership_df = gun_ownership_df.join(all_state_crime_2014, how="outer")
except:
    pass

# Remove bad columns
gun_ownership_df = gun_ownership_df.loc[:, ~gun_ownership_df.columns.str.contains('^Unnamed')]


In [ ]:
# dataPoints = ['Violent crime total']
# for col in dataPoints:
#     dat = []
#     for stateName in gun_ownership_df.index:
#         if stateName in crime_by_state:
#             dat.append(crime_by_state[stateName].tail(1)[col].values[0])
#         else:
#             dat.append(None)
#     gun_ownership_df[col] = dat
gun_ownership_df.head(60)

In [ ]:
# Teammate Chris code ends here

In [ ]:
# Teammate Matt code begins here

In [ ]:
# Teammate Matt code ends here

In [ ]:
# Teammate Carrie code begins here

In [ ]:
# Teammate Carrie code ends here

In [ ]:
# Teammate Glenda code begins here

In [ ]:
gun_ownership_per_capita_df = gun_ownership_df
gun_ownership_per_capita_df = gun_ownership_per_capita_df.reset_index(drop=False)
gun_ownership_per_capita_df = gun_ownership_per_capita_df[["State","# of guns per capita"]]
gun_ownership_per_capita_df.head()

In [ ]:
all_state_crime_2014_df = all_state_crime_2014
all_state_crime_2014_df = all_state_crime_2014_df.reset_index(drop=False)
all_state_crime_2014_df.head()

In [ ]:
murder_manslaughter_2014_df = all_state_crime_2014_df[["State","Murder and nonnegligent manslaughter rate"]]
murder_manslaughter_2014_df.head()

In [ ]:
aggravated_assault_2014_df = all_state_crime_2014_df[["State","Aggravated assault rate"]]
aggravated_assault_2014_df.head()

In [ ]:
vehicle_theft_2014_df = all_state_crime_2014_df[["State","Motor vehicle theft rate"]]
vehicle_theft_2014_df.head()

In [ ]:
robbery_2014_df = all_state_crime_2014_df[["State","Robbery rate"]]
robbery_2014_df.head()

In [ ]:
burglary_2014_df = all_state_crime_2014_df[["State","Burglary rate"]]
burglary_2014_df.head()

In [ ]:
import plotly.express as px

combined_data_df = pd.merge(burglary_2014_df,gun_ownership_per_capita_df,on="State")

import plotly.express as px

fig = px.scatter(combined_data_df,
                 x="Burglary rate", 
                 y="# of guns per capita",
                 size="Burglary rate", 
                 color="State",
                 hover_name="State",
                 title="Burglary Rate in the US"
                )
fig.show()

In [ ]:
import plotly.express as px

combined_data_df = pd.merge(robbery_2014_df,gun_ownership_per_capita_df,on="State")

import plotly.express as px

fig = px.scatter(combined_data_df,
                 x="Robbery rate", 
                 y="# of guns per capita",
                 size="Robbery rate", 
                 color="State",
                 hover_name="State",
                 title="Robbery Rate in the US"
                )
fig.show()

In [ ]:
import plotly.express as px

combined_data_df = pd.merge(vehicle_theft_2014_df,gun_ownership_per_capita_df,on="State")

import plotly.express as px

fig = px.scatter(combined_data_df,
                 x="Motor vehicle theft rate", 
                 y="# of guns per capita",
                 size="Motor vehicle theft rate", 
                 color="State",
                 hover_name="State",
                 title="Motor Behicle Theft Rate in the US"
                )
fig.show()

In [ ]:
import plotly.express as px

combined_data_df = pd.merge(aggravated_assault_2014_df,gun_ownership_per_capita_df,on="State")

import plotly.express as px

fig = px.scatter(combined_data_df,
                 x="Aggravated assault rate", 
                 y="# of guns per capita",
                 size="Aggravated assault rate", 
                 color="State",
                 hover_name="State",
                 title="Agravated Assault Rate in the US"
                )
fig.show()

In [ ]:
import plotly.express as px

combined_data_df = pd.merge(murder_manslaughter_2014_df,gun_ownership_per_capita_df,on="State")

import plotly.express as px

fig = px.scatter(combined_data_df,
                 x="Murder and nonnegligent manslaughter rate", 
                 y="# of guns per capita",
                 size="Murder and nonnegligent manslaughter rate", 
                 color="State",
                 hover_name="State",
                 title="Murder and Manslaughter Rate in the US"
                )
fig.show()

In [ ]:
murder_manslaughter_2014_df.head()

In [ ]:
# Teammate Glenda code ends here